## Generating data for ML

In [25]:
import pandas as pd
from collections import Counter
import random 
import ultimate
# define card set
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
rank_values = {rank: i for i, rank in enumerate(ranks, start=2)}

deck = [{'rank': rank, 'suit': suit} for suit in suits for rank in ranks]

budget = 1000
combinations = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Four of a Kind", 
                "Full House", "Straight", "Flush", "Straight Flush", "Royal Flush"]
combinations_values = {combination: i for i, combination in enumerate(combinations, start=1)}
# set ordered winning combinations
winning_hands = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", "Flush", 
                "Full House", "Four of a Kind", "Straight Flush", "Royal Flush"]
#enumerate the deck
enumerated_deck = dict(enumerate(deck, start=1))
num_deck = list(range(1, 53))

In [32]:
# ante is 1, blind is 1, player bets 1 or 0 (return bet or 
# fold depending whether you will win or lose)
def calculate_output_for_third_round(row):
    
    player_hand = [enumerated_deck[card] for card in row[1:6]]
    dealer_hand = [enumerated_deck[card] for card in row[2:]]

    #print(player_hand)
    #print(dealer_hand)

    player_combination = ultimate.get_best_hand(player_hand)
    dealer_combination = ultimate.get_best_hand(dealer_hand)
    
    if winning_hands.index(player_combination) > winning_hands.index(dealer_combination):
        return "Bet"
    elif winning_hands.index(player_combination) == winning_hands.index(dealer_combination):
        result = ultimate.decider(player_combination, player_hand, 
                                  dealer_combination, dealer_hand)
        if result == "Player":
            return "Bet"
        elif result == "Dealer":	
            return "Fold"
        else:
            return "Bet"	# need to decide about this
    else:
        return "Fold"
    


In [47]:
# full dataset
hands = 10
multiplier = 10

header = ["C1", "C2", "R1", "R2", "R3", "R4", "R5", "D1", "D2"]
full_data_set = pd.DataFrame(columns=header)
for A in range(hands):
    num_deck_copy = num_deck.copy()
    player = [num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1)), 
              num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1))]
    dealer = [num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1)), 
              num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1))]
    for K in range(multiplier):
        num_deck_copy_copy = num_deck_copy.copy()	
        flop = [num_deck_copy_copy.pop(random.randint(0, len(num_deck_copy_copy) - 1)), 
                num_deck_copy_copy.pop(random.randint(0, len(num_deck_copy_copy) - 1)), 
                num_deck_copy_copy.pop(random.randint(0, len(num_deck_copy_copy) - 1))]
        for K2 in range(multiplier):
            num_deck_copy_copy_copy = num_deck_copy_copy.copy()
            river = [num_deck_copy_copy_copy.pop(random.randint(0, len(num_deck_copy_copy_copy) - 1)),
                     num_deck_copy_copy_copy.pop(random.randint(0, len(num_deck_copy_copy_copy) - 1))]
            row = player + flop + river + dealer
            full_data_set.loc[len(full_data_set)] = row
print("generated variables")
data_for_third = full_data_set.copy()
data_for_third["Y"] = full_data_set.apply(calculate_output_for_third_round, axis=1)
pd.set_option('display.max_rows', 20)  # Show up to 100 rows
data_for_second = full_data_set[::10].drop(columns=["R4", "R5"]) #get without last two cards
data_for_first = data_for_second[::10].drop(columns=["R1", "R2", "R3"])  #get only first two cards
data_for_first

generated variables


,C1,C2,D1,D2
0,26,12,17,46
100,31,50,14,34
200,37,25,39,3
300,45,20,43,40
400,25,2,13,1
500,4,8,28,52
600,35,23,38,11
700,28,4,29,46
800,41,11,6,36
900,4,25,13,37
